# Tutorial: Instalação e configuração do OAI 5G Core Netwrok

# Parte 01: Instalação do OAI 5G CN 

## Passo 1: Instalar dependências

Primeiramente, deve-se instalar as dependências do OAI 5G CN. Em um terminal, execute os seguintes comandos:

```bash
sudo apt-get install ca-certificates

sudo apt-get install curl 

sudo apt-get install gnupg

sudo apt-get install lsb-release
```

## Passo 2: Habilitar o "packet forwarding"

É necessário habilitar a configuração \textit{"packet forwarding"}, a qual será importante para a conectividade fim-a-fim. Execute os seguintes comandos:

```bash
sudo sysctl net.ipv4.conf.all.forwarding=1

sudo iptables -P FORWARD ACCEPT 
```


## Passo 3: Instalar o Docker e o Docker-compose

Execute os comandos:

```bash
sudo rm /etc/apt/sources.list.d/docker.list*

sudo mkdir -p /etc/apt/keyrings 

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(. /etc/os-release; echo "$UBUNTU_CODENAME") stable"
```
```bash
sudo chmod a+r /etc/apt/keyrings/docker.gpg

sudo apt-get update

sudo apt-get install docker-ce docker-ce-cli containerd.io docker-compose-plugin

sudo curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose

sudo chmod +x /usr/local/bin/docker-compose
```

## Passo 4: Fazer o "pull" das imagens do Docker Hub

Cada função virtual de rede (VNF), é executada em um container Docker. Neste passo será realizado o "pull" das imagens das VNFs. Execute os seguintes comandos:

```bash
sudo docker pull oaisoftwarealliance/oai-amf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-nrf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-smf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udr:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udm:v1.5.0

sudo docker pull oaisoftwarealliance/oai-ausf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-spgwu-tiny:v1.5.0

sudo docker pull oaisoftwarealliance/trf-gen-cn5g:latest
```

## Passo 5: Atribuir uma tag para cada imagem

Execute os comandos:

```bash
sudo docker image tag oaisoftwarealliance/trf-gen-cn5g:latest trf-gen-cn5g:latest

sudo docker image tag oaisoftwarealliance/oai-amf:v1.5.0 oai-amf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-nrf:v1.5.0 oai-nrf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-smf:v1.5.0 oai-smf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udr:v1.5.0 oai-udr:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udm:v1.5.0 oai-udm:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-ausf:v1.5.0 oai-ausf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-spgwu-tiny:v1.5.0 oai-spgwu-tiny:v1.5.0
```

## Passo 6: Clonar o repositório do OAI 5G CN

Execute os comandos:

```bash
git clone https://gitlab.eurecom.fr/oai/cn5g/oai-cn5g-fed.git

cd oai-cn5g-fed

git checkout -f v1.5.0
```

# Parte 02: Configuração do OAI 5G CN

## Passo 1: Baixar arquivo de configuração

A configuração é realizada utilizando docker-compose. Para baixar o arquivo e já colocá-lo na pasta correta (PATH_TO/oai-cn5g-fed/docker-compose/), execute o comando abaixo: 

```bash
wget -O PATH_TO/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml https://github.com/matheusfagundess/oai-flexric-installation/raw/main/arquivos/docker-compose-basic-nrf.yaml
```

## Passo 2: Baixar banco de dados

Para baixar o arquivo de banco de dados utilizado nesse deployment e colocá-lo na pasta correta (PATH_TO/oai-cn5g-fed/docker-compose/database/) execute o comando abaixo:

```bash
wget -O PATH_TO/oai-cn5g-fed/docker-compose/database/oai_db.sql https://github.com/matheusfagundess/oai-flexric-installation/raw/main/arquivos/oai_db.sql
```

## Passo 3: Sincronização de componentes

Para sincronizar os componentes, execute os comandos abaixo:

```bash
cd PATH_TO/oai-cn5g-fed/

sudo ./scripts/syncComponents.sh

sudo git submodule deinit --all --force

sudo git submodule init

sudo git submodule update
```

## Passo 4: Criar bridge 5GC

É necessário criar uma subnet para habilitar a comunicação entre os containers Docker, ou seja, entre as funções virtuais de rede. Para isso, execute o comando abaixo:

```bash
sudo docker network create --driver=bridge --subnet=192.168.70.128/26 -o "com.docker.network.bridge.name"="demo-oai" demo-oai-public-net
```